# TAP viewer

This notebook is for inspecting timelapse microscopy data, with associated sinhgle-cell labels and tracks, showing the infection of human macrophages with Mycobacterium Tuberculosis (Mtb), acquired on an Opera Phenix confocal microscope. 

In [1]:
import napari
from macrohet import dataio, tile, visualise
import btrack
print(btrack.__version__)
import os, glob
import napari_animation
import dask.array as da

0.6.1


# Load image

In [3]:
images = da.from_zarr('/mnt/DATA/macrohet/macrohet_images/ND0000/zarr/2.10_gfp_rfp.zarr')

In [4]:
# do a max projection
images = da.max(images, axis = 2)

In [5]:
images

,Array,Chunk
Bytes,55.01 GB,146.31 MB
Shape,"(376, 2, 6048, 6048)","(1, 2, 6048, 6048)"
Count,1129 Tasks,376 Chunks
Type,uint16,numpy.ndarray


In [6]:
%%time
images_loaded = images.compute()

CPU times: user 4min 25s, sys: 14min 27s, total: 18min 52s
Wall time: 3min 43s


# Launch napari

In [7]:
viewer = napari.Viewer()

viewer.add_image(images_loaded, channel_axis=1)

[<Image layer 'Image' at 0x7f72c41dea90>,
 <Image layer 'Image [1]' at 0x7f731a7bc0d0>]

# Load prelim tracks

In [11]:
import btrack

In [12]:
with btrack.io.HDF5FileHandler('/mnt/DATA/macrohet/labels/ND0000/dev/ND0000_2_10_cellpose_default_tracks.h5', 
                                       'r', 
                                           obj_type='obj_type_1'#'obj_type_2'
                                       ) as reader:
            # tracks = [t for t in reader.tracks if np.mean(t.properties['area']) > 1000]
            tracks = reader.tracks #[t for t in reader.tracks if len(t) >= 10] #

[INFO][2023/11/02 01:30:04 pm] Opening HDF file: /mnt/DATA/macrohet/labels/ND0000/dev/ND0000_2_10_cellpose_default_tracks.h5...
INFO:btrack.io.hdf:Opening HDF file: /mnt/DATA/macrohet/labels/ND0000/dev/ND0000_2_10_cellpose_default_tracks.h5...
[INFO][2023/11/02 01:30:04 pm] Loading tracks/obj_type_1
INFO:btrack.io.hdf:Loading tracks/obj_type_1
[INFO][2023/11/02 01:30:04 pm] Loading LBEP/obj_type_1
INFO:btrack.io.hdf:Loading LBEP/obj_type_1
[INFO][2023/11/02 01:30:04 pm] Loading objects/obj_type_1 (283308, 5) (283308 filtered: None)
INFO:btrack.io.hdf:Loading objects/obj_type_1 (283308, 5) (283308 filtered: None)
[INFO][2023/11/02 01:30:08 pm] Closing HDF file: /mnt/DATA/macrohet/labels/ND0000/dev/ND0000_2_10_cellpose_default_tracks.h5
INFO:btrack.io.hdf:Closing HDF file: /mnt/DATA/macrohet/labels/ND0000/dev/ND0000_2_10_cellpose_default_tracks.h5


In [13]:
napari_tracks, _, _ = btrack.utils.tracks_to_napari([t for t in tracks if len(t) > 10], ndim = 2)

In [15]:
viewer.add_tracks(napari_tracks, scale = (5.04, 5.04))

<Tracks layer 'napari_tracks' at 0x7f730f02deb0>

In [17]:
from napari_animation import Animation

animation = Animation(viewer)

viewer.dims.ndisplay = 3
viewer.camera.angles = (0.0, 0.0, 90.0)
animation.capture_keyframe()
viewer.camera.zoom = 2.4
animation.capture_keyframe()
viewer.camera.angles = (-7.0, 15.7, 62.4)
animation.capture_keyframe(steps=60)
viewer.camera.angles = (2.0, -24.4, -36.7)
animation.capture_keyframe(steps=60)
viewer.reset_view()
viewer.camera.angles = (0.0, 0.0, 90.0)
animation.capture_keyframe()
animation.animate('demo.mov', canvas_only=False)

ImportError: cannot import name '__version__' from 'napari._version' (/home/dayn/analysis/miniforge3/envs/tap/lib/python3.8/site-packages/napari/_version.py)

In [19]:
# from napari._version import __version__ as napari_version
napari.__version__

'0.4.11'

# Loading example data set

In [14]:
import tarrow

In [5]:
model = tarrow.models.TimeArrowNet.from_folder(model_folder='/home/dayn/analysis/models/tarrow/reduced_batchsize/macro_backbone_unet/')

In [6]:
x = torch.rand(1, 2, 1, 96, 96)

In [7]:
x.shape

torch.Size([1, 2, 1, 96, 96])

In [ ]:
rep = model.embedding(x)

In [9]:
print(f"Dense representations for image `x` with shape {tuple(rep.shape)}")

Dense representations for image `x` with shape (1, 2, 32, 96, 96)
